### Importing packages and data_utils.py

In [1]:
from climsim_utils.data_utils import *

2023-08-18 01:23:35.082885: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-18 01:23:36.060440: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
!pwd

/ocean/projects/atm200007p/jlin96/neurips_proj/ClimSim_release/evaluation


### Load in inputs and outputs for ground truth

In [3]:
# Replace these paths with corresponding paths on your machine
input_path = '/ocean/projects/atm200007p/jlin96/neurips_proj/e3sm_train_npy/val_input_stride6.npy'
target_path = '/ocean/projects/atm200007p/jlin96/neurips_proj/e3sm_train_npy/val_target_stride6.npy'

In [4]:
x_true = np.load(input_path).astype(np.float64)
y_true = np.load(target_path).astype(np.float64)

### Instantiate data_utils class

In [5]:
data_path = '/ocean/projects/atm200007p/walrus/for_jerry/train/'
grid_path = '/ocean/projects/atm200007p/jlin96/neurips_proj/ClimSim_release/grid_info/ClimSim_low-res_grid-info.nc'
norm_path = '/ocean/projects/atm200007p/jlin96/neurips_proj/ClimSim_release/normalizations/'

grid_info = xr.open_dataset(grid_path)
inp_mean = xr.open_dataset(norm_path + 'inputs/input_mean.nc')
inp_max = xr.open_dataset(norm_path + 'inputs/input_max.nc')
inp_min = xr.open_dataset(norm_path + 'inputs/input_min.nc')
out_scale = xr.open_dataset(norm_path + 'outputs/output_scale.nc')

scoring_data = data_utils(data_path = data_path,
                          grid_info = grid_info, 
                          inp_mean = inp_mean, 
                          inp_max = inp_max, 
                          inp_min = inp_min, 
                          out_scale = out_scale)

In [6]:
# set inputs and outputs to V1 subset
scoring_data.set_to_v1_vars()
# load target output
scoring_data.target_scoring = np.load(target_path)
# set pressure grid
scoring_data.set_pressure_grid(x_true)

### Load in model predictions

In [7]:
cvae_pred_path = '/ocean/projects/atm200007p/shared/neurips_proj/final_metrics/predictions/cVAE/cvae_preds_manual.h5'
hsr_pred_path = '/ocean/projects/atm200007p/shared/neurips_proj/final_metrics/predictions/HSR/hsr_preds_bestcrps.h5'
rpn_pred_path = '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/rpn_pred_v1_stride6.npy'
cnn_pred_path = '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/val_predict_cnn_reshaped_stride6_FINAL.npy'
mlp_pred_path = '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/001_backup_phase-7_retrained_models_step2_lot-147_trial_0027.best.h5.npy'

In [8]:
scoring_data.model_names = ['cVAE','HSR','RPN','CNN', 'MLP']
preds = [scoring_data.load_h5_file(load_path = cvae_pred_path), 
         scoring_data.load_h5_file(load_path = hsr_pred_path), 
         scoring_data.load_npy_file(load_path = rpn_pred_path), 
         scoring_data.load_npy_file(load_path = cnn_pred_path), 
         scoring_data.load_npy_file(load_path = mlp_pred_path)]
scoring_data.preds_scoring = dict(zip(scoring_data.model_names, preds))

### Weight and rescale simulation targets

In [9]:
targets = scoring_data.output_weighting(y_true)

### Weight and rescale model outputs

In [10]:
preds_cVAE = scoring_data.output_weighting(scoring_data.preds_scoring['cVAE'])

preds_HSR = scoring_data.output_weighting(scoring_data.preds_scoring['HSR'])

preds_RPN = scoring_data.output_weighting(scoring_data.preds_scoring['RPN'])

preds_CNN = scoring_data.output_weighting(scoring_data.preds_scoring['CNN'])

preds_MLP = scoring_data.output_weighting(scoring_data.preds_scoring['MLP'])

In [11]:
scoring_data.calc_MAE(preds_MLP['heating'], targets['heating'])

array([2.48810441e-03, 4.99724094e-03, 8.76758175e-03, 1.47557543e-02,
       2.30482093e-02, 3.34678148e-02, 5.08360518e-02, 6.79574945e-02,
       7.78484613e-02, 9.34334972e-02, 1.10891516e-01, 1.27929635e-01,
       1.32231286e-01, 1.45734283e-01, 1.64263233e-01, 1.96093256e-01,
       2.47765425e-01, 4.04718618e-01, 7.13249399e-01, 1.05959364e+00,
       1.09819824e+00, 1.19503676e+00, 1.35142794e+00, 1.51855179e+00,
       1.67193697e+00, 1.83208818e+00, 2.05268240e+00, 2.28865263e+00,
       2.45375703e+00, 2.66744006e+00, 2.98282797e+00, 3.28212054e+00,
       3.56680301e+00, 3.93279688e+00, 4.31148096e+00, 4.68898232e+00,
       5.06267101e+00, 5.44454106e+00, 5.71696143e+00, 5.90012906e+00,
       6.00360151e+00, 6.07438613e+00, 6.17479732e+00, 6.16412421e+00,
       6.18342603e+00, 6.14224057e+00, 5.98347202e+00, 5.69510284e+00,
       5.35002381e+00, 4.89994824e+00, 4.50862157e+00, 4.15251703e+00,
       3.82127525e+00, 3.55926722e+00, 3.35267995e+00, 3.19454866e+00,
      

In [ ]:
preds_MLP['heating'].shape

In [ ]:
heating.shape

In [ ]:
heating

In [ ]:
#CRPS paths
hsr_path_crps = '/ocean/projects/atm200007p/shared/neurips_proj/final_metrics/predictions/HSR/hsr_samples_bestcrps.h5'
cvae_path_crps = '/ocean/projects/atm200007p/shared/neurips_proj/final_metrics/predictions/cvae_samples_bestcrps.h5'
rpn_path_crps = '/ocean/projects/atm200007p/shared/neurips_proj/final_metrics/predictions/RPN/samples_test_H_stride_6_all.npy'

In [ ]:
hsr_samples = data_utils.load_h5_file(hsr_path_crps)


In [ ]:
cvae_samples = data_utils.load_h5_file(cvae_path_crps)

In [ ]:
rpn_samples = data_utils.load_npy_file(rpn_path_crps)